In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import xlrd
import os 
import sqlite3
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
from tabulate import tabulate
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
from scipy.stats import norm
import statsmodels.api as sm
from sklearn.impute import SimpleImputer

sns.set(style="whitegrid")

### Functions:

In [ ]:
def freeman_diaconis( data):
    quartiles = stats.mstats.mquantiles( data, [0.25, 0.5, 0.75])
    iqr = quartiles[2] - quartiles[ 0]
    n = len( data)
    h = 2.0 * (iqr/n**(1.0/3.0))
    return int( h)

In [ ]:
def classify_correlation(r):
    r = abs(r)
    if r < 0.16:
        return "very weak"
    if r < 0.29:
        return "weak"
    if r < 0.49:
        return "low"
    if r < 0.69:
        return "moderate"
    if r < 0.89:
        return "strong"
    return "very strong"

In [ ]:
def correlation(data, x, y):
    print("Correlation coefficients:")
    r = stats.pearsonr(data[x], data[y])[0]
    print( "r   =", r, f"({classify_correlation(r)})")
    rho = stats.spearmanr(data[x], data[y])[0]
    print( "rho =", rho, f"({classify_correlation(rho)})")

In [ ]:
def describe_by_category(data, numeric, categorical, transpose=False):
    grouped = data.groupby(categorical)
    grouped_y = grouped[numeric].describe()
    if transpose:
        print(grouped_y.transpose())
    else:
        print(grouped_y)

In [ ]:
def lowess_scatter(data, x, y, jitter=0.0, skip_lowess=False):

    if skip_lowess:
        fit = np.polyfit(data[x], data[y], 1)
        line_x = np.linspace(data[x].min(), data[x].max(), 10)
        line = np.poly1d(fit)
        line_y = list(map(line, line_x))
    else:
        lowess = sm.nonparametric.lowess(data[y], data[x], frac=.3)
        line_x = list(zip(*lowess))[0]
        line_y = list(zip(*lowess))[1]

    figure = plt.figure(figsize=(10, 6))

    axes = figure.add_subplot(1, 1, 1)

    xs = data[x]
    if jitter > 0.0:
        xs = data[x] + stats.norm.rvs( 0, 0.5, data[x].size)

    axes.scatter(xs, data[y], marker="o", color="DimGray", alpha=0.5)
    axes.plot(line_x, line_y, color="DarkRed")

    title = "Plot of {0} v. {1}".format(x, y)
    if not skip_lowess:
        title += " with LOWESS"
    axes.set_title(title)
    axes.set_xlabel(x)
    axes.set_ylabel(y)

    plt.show()
    plt.close()

In [ ]:
db_path = '../datasets/datawarehouse.db'

#### ETL - Price (for some EDA instances), host_response_rate, and host_acceptance_rate

Load the data from the `listings` table:

In [ ]:
data = pd.read_sql_query("SELECT * FROM listings", sqlite3.connect(db_path))

In [ ]:
eda_dkn = data.loc[:,["price", "host_response_rate", "host_acceptance_rate"]]
eda_dkn.head()

We then convert categorical variables `price`, `host_response_rate`, and `host_acceptance_rate` into numerical variables.

We first remove `$` and `,` from the column of `price` and `%` from the columns of `host_response_rate` and `host_acceptance_rate`:

In [ ]:
eda_dkn['price'] = eda_dkn['price'].str.replace('$', '')
eda_dkn['price'] = eda_dkn['price'].str.replace(',', '')
eda_dkn['host_response_rate'] = eda_dkn['host_response_rate'].str.replace('%', '')
eda_dkn['host_acceptance_rate'] = eda_dkn['host_acceptance_rate'].str.replace('%', '')
eda_dkn.head()

We then look at how many `N/A`s are in these columns:

In [ ]:
eda_dkn['price'].str.count('N/A').sum()

In [ ]:
eda_dkn['host_response_rate'].str.count('N/A').sum()

In [ ]:
eda_dkn['host_acceptance_rate'].str.count('N/A').sum()

We then convert `host_response_rate` and `host_acceptance_rate` to floats:

In [ ]:
eda_dkn['host_response_rate'] = eda_dkn['host_response_rate'].str.replace('N/A', '')
eda_dkn['host_acceptance_rate'] = eda_dkn['host_acceptance_rate'].str.replace('N/A', '')
eda_dkn.head()

In [ ]:
for price in eda_dkn.columns:
    eda_dkn[price] = pd.to_numeric(eda_dkn[price], errors='coerce')

In [ ]:
imputer1 = SimpleImputer(strategy='mean', missing_values=np.nan)
imputer1 = imputer1.fit(eda_dkn[['host_response_rate']])
eda_dkn['host_response_rate'] = imputer1.transform(eda_dkn[['host_response_rate']])

imputer2 = imputer1.fit(eda_dkn[['host_acceptance_rate']])
eda_dkn['host_acceptance_rate'] = imputer2.transform(eda_dkn[['host_acceptance_rate']])
eda_dkn.head()

Now we will add the categorical variable `host_response_time` to the dataframe:

In [ ]:
eda_dkn['host_response_time'] = data['host_response_time']
eda_dkn.head()

Now we will replace missing values in `host_response_time` with the most frequent value:

In [ ]:
eda_dkn['host_response_time'] = eda_dkn['host_response_time'].str.replace('N/A', '')
eda_dkn.head()

In [ ]:
imputer3 = SimpleImputer(missing_values=None, strategy='constant', fill_value='within an hour')
imputer3 = imputer3.fit(eda_dkn[['host_response_time']])
eda_dkn[['host_response_time']] = imputer3.transform(eda_dkn[['host_response_time']])
eda_dkn.head()

We decided not to fill in the missing values in host_response_time, since it will bring up bias. 

### Target Variable: Price of AirBnB Listing

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT id, CAST(REPLACE(price, '$', '') AS FLOAT) as avg_price
FROM listings;
'''

cur.execute(query)
con.commit()

price_arr = cur.fetchall()
con.close()

In [ ]:
cost_arr = []
for pairs in price_arr:
    cost_arr.append(pairs[1])

In [ ]:
figure = plt.figure(figsize=(10, 6)) # first element is width, second is height.

axes = figure.add_subplot(1, 1, 1)
axes.hist(cost_arr, color="dimgray", bins=20)
axes.set_ylabel( "Count")
axes.set_xlabel( "Cost ($)")
axes.set_title( "Average Cost of an AirBnB Listing in Washington, DC")

plt.show()
plt.close()

## Single Variable EDA

### Neighborhood Groups

The neighborhood group variable is a categorical value. In Washington D.C., there are [131 neighborhoods](https://en.wikipedia.org/wiki/Neighborhoods_in_Washington,_D.C.) that are unofficially defined. The AirBnB dataset compiles neighborhood names using the city's defined borders for the neighborhoods, along with the latitude and longitude of the property listing. 

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT neighbourhood_cleansed, COUNT(*) as neighborhood_count
FROM listings
GROUP BY neighbourhood_cleansed
ORDER BY neighborhood_count DESC;
'''

cur.execute(query)
con.commit()

neighborhood = cur.fetchall()
con.close()

# Convert to DataFrame so Pandas functions can be used for EDA
nbhd = []
n_count = []
for n in neighborhood:
    nbhd.append(n[0])
    n_count.append(n[1])
n_df = pd.DataFrame({'nbhd': nbhd,
                    'n_count': n_count})

In [ ]:
n_df.sort_values(by='n_count', ascending=False)
t = tabulate(n_df, 
             headers=('Index', 'Neighborhood Grouping', 'Count'),
            tablefmt='fancy_grid')
print('     Count of AirBnB Listings, By Neighborhood')
print(t)
print('\n')

In this dataset, the neighborhoods are grouped based on proximity. There are 39 unique neighborhood groupings, reduced from the 131 individual neighborhoods in Washington, D.C. The neighborhood groupings with the greatest number of listings are: 
1. Capitol Hill, Union Park (561 listings)
2. Union Station, Stanton Park, Kingman Park (550 listings)
3. Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View (456 listings)

The Captiol Hill neighborhood is one of the most densely populated in Washington D.C $^{[1]}$. The Union Station neighborhood is near Washington Union Station, which is Amtrack's second-busiest station $^{[2]}$. The Columbia Heights and Mt. Pleasant areas are located close to a major University, and contains many restaurants $^{[3]}$.

*Sources*
1. https://en.wikipedia.org/wiki/Capitol_Hill
2. https://en.wikipedia.org/wiki/Washington_Union_Station
3. https://en.wikipedia.org/wiki/Columbia_Heights_(Washington,_D.C.)

### Neighborhood Groups - Tax Assessed Property Values Dataset

The secondary dataset on tax-assessed property values contains neighborhood names. The same neighborhood grouping were used in both the primary and secondary dataset. 

For the secondary datasource, any properties with a tax-assessed value of greater than 5 million USD, or less than 50 thousand USD, will be excluded. The tax-assessed value can be considered to be similar to the nightly cost of an AirBnB. Comparing the tax-assessed value by neighborhood could indicate which neighborhoods of DC are more expensive, and may also contain more expensive AirBnb listings.

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT NBHDNAME, COUNT(*) as neighborhood_count
FROM property_info
WHERE PROPTYPE LIKE '%Residential%'
GROUP BY NBHDNAME
ORDER BY neighborhood_count DESC;
'''

cur.execute(query)
con.commit()
prop_info = cur.fetchall()

In [ ]:
nbhd = []
count = []
for vals in prop_info:
    nbhd.append(vals[0])
    count.append(vals[1])
    
prop_df = pd.DataFrame({'nbhd': nbhd,
                       'count': count})

prop_df.sort_values(by='count', ascending=False)
t = tabulate(prop_df, 
             headers=('Index', 'Neighborhood Grouping', 'Count'),
            tablefmt='fancy_grid')
print('     Number of Properties In Neighborhoods')
print(t)
print('\n')

The three neighborhoods with the highest number of residential properties are:
1. Capitol Hill, Lincoln Park   
2. Dupont Circle, Connecticut Avenue/K Street
3. Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View  

The "Capitol Hill, Lincoln Park" and "Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View" neighborhood groupings fall in the top three, based on count of properties, for both datasets. In the property assessed dataset, the "Union Station, Staton Park, Kingman Park" grouping ranks 29th by property count, while in the AirBnB data, it ranked second by count. The "Dupont Circle, Connecticut Avenue/K Street" grouping has the fourth highest count of AirBnB listings, and ranks second, by count, in the property assessed values.

The property assessed values dataset also includes entire apartment buildings. It may be the case that Union Station contains more group living buildings, which would limit the overall count of individual properties in the secondary dataset, since an entire apartment building may only count as one property. 

### Number of Bathrooms

The number of bathrooms in an AirBnB is a categorical value, since AirBnbs can have "shared" or "private" bathrooms. Additionally, there may be "half-baths" as well. 

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT bathrooms_text, COUNT(*) as num_bath
FROM listings
GROUP BY bathrooms_text
ORDER BY num_bath DESC;
'''

cur.execute(query)
con.commit()

bathroom = cur.fetchall()
con.close()

In [ ]:
bath_type = []
bath_count = []
for vals in bathroom:
    bath_type.append(vals[0])
    bath_count.append(vals[1])
    
bath_df = pd.DataFrame({'bath_type': bath_type,
                       'count': bath_count})

In [ ]:
bath_df.sort_values(by='count', ascending=False)
bath_df.set_index('bath_type', inplace=True)
t = tabulate(bath_df, 
             headers=('Bathroom Type', 'Count'),
            tablefmt='fancy_grid')
print('     Counts of Bathroom Types In AirBnb')
print(t)
print('\n')

The most common bathroom type for Airbnb is "1 bath". Some listings don't have bath populated: these can be filled with the most common bathroom type, which is one bath. It may make sense to reduce the number of "types of bath" categories. This "bathroom_dict" variable can be used later in pairwise analysis.

In [ ]:
'''
Limit the number of bath categories
'''
bathroom_dict = {}
bathroom_mapping = {}

def getNewBathroomCat(s):
    str_elements = s.split(' ')
    bath_cat = ''
    if str_elements[0] == '':
        bath_cat = '1 Private'
    elif str_elements[0] == 'Half-bath':
        bath_cat = '1 Private'
    elif str_elements[0] == '0':
        bath_cat = '1 Private'
    else:
        try:
            num_bath = float(str_elements[0])
            if num_bath >= 3:
                bath_cat = '3+'
            elif num_bath >= 2:
                bath_cat = '2'
            else:
                bath_cat = '1'
            if str_elements[1] == 'shared':
                bath_cat += ' Shared'
            else:
                bath_cat += ' Private'
        except:
            if str_elements[0] == 'Private':
                bath_cat = '1 Private'
            elif str_elements[0] == 'Shared':
                bath_cat = '1 Shared'
    return bath_cat
    
for ind in range(0, len(bath_type)):
    s = bath_type[ind]
    c = bath_count[ind]
    
    b_type = getNewBathroomCat(s)
    bathroom_mapping[s] = b_type

    if b_type not in bathroom_dict:
        bathroom_dict[b_type] = c
    else:
        bathroom_dict[b_type] +=c

In [ ]:
lim_bath_df = pd.DataFrame({'bath_type': bathroom_dict.keys(), 'count': bathroom_dict.values()})

lim_bath_df.sort_values(by='count', ascending=False)
lim_bath_df.set_index('bath_type', inplace=True)
t = tabulate(lim_bath_df, 
             headers=('Bathroom Type', 'Count'),
            tablefmt='fancy_grid')
print('     Counts of Bathroom Types In AirBnb')
print(t)
print('\n')

### "Is Superhost" query

Some hosts on AirBnB are marked as Superhosts. This is a specific Airbnb program that "celebrates and rewards Airbnb's top-rated and most experienced hosts". These hosts must meet the following critera:
- 4.8+ overall rating
- 10+ stays
- <1% cancellation rate
- 90% response rate

This is a categorical, boolean value, with the host either being labeled as a superhost, or not being labeled as a superhost.

*Sources:*
1. https://www.airbnb.com/d/superhost

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT host_is_superhost, COUNT(*) as num_superhosts
FROM listings
GROUP BY host_is_superhost
ORDER BY num_superhosts DESC;
'''

cur.execute(query)
con.commit()

superhost = cur.fetchall()

con.close()

In [ ]:
s_status = []
count = []
for vals in superhost:
    s_status.append(vals[0])
    count.append(vals[1])
    
super_df = pd.DataFrame({'s_status': s_status,
                       'count': count})

super_df.set_index('s_status', inplace=True)
t = tabulate(super_df, 
             headers=('Is Superhost', 'Count'),
            tablefmt='fancy_grid')
print('     Number of Properties In Neighborhoods')
print(t)
print('\n')

In the AirBnB listings, 4496 of the listings have hosts that are not superhosts. This is about 70% of all listings. The other 30%, or 1972 listings, are labeled as superhosts. Five of the points in the dataset are not labeled, and thuse should be treated as `is_superhost=False`.

### Bedrooms
The bedrooms variable is a numerical integer value. The `beds` variable represents the number of beds in the AirBnb. More beds generally corresponds to a higher capacity for overnight guests.

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT id, CAST(bedrooms AS INT), CAST(beds AS INT), CAST(accommodates AS INT), CAST(REPLACE(price, '$', '') AS FLOAT)
FROM listings;'''

cur.execute(query)
con.commit()

b_arr = cur.fetchall()

In [ ]:
b_arr[0]

The SQL query returns a list of tuple objects, with:
- `b_arr[0]`: listing_id
- `b_arr[1]`: bedrooms
- `b_arr[2]`: beds
- `b_arr[3]`: accommodates
- `b_arr[4]`: price

In [ ]:
listing_id = []
bedrooms = []
beds = []
accommodates = []
price = []
for b in b_arr:
    listing_id.append(b[0])
    bedrooms.append(b[1])
    beds.append(b[2])
    accommodates.append(b[3])
    price.append(b[4])
    
b_df = pd.DataFrame({'listing_id': listing_id,
                    'bedrooms': bedrooms,
                    'beds': beds,
                    'accommodates': accommodates,
                    'price': price})

b_df.info()

The variables `beds`, `bedrooms`, and `accommodates` are all integer values. The `price` variable is a float value. 

In [ ]:
b_df.beds.describe()

75% of AirBnB listings in the DC area contain two or fewer bedrooms. There is one AirBnB that has a maximum number of bedrooms of 15. 

In [ ]:
h=1
mn = int(b_df["bedrooms"].min())
mx = int(b_df["bedrooms"].max())
bins = [i for i in range( mn, mx, h)]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.hist(b_df["bedrooms"],bins=bins, histtype='bar', color="DimGray", ec='black')
axes.set_title("Bedrooms distribution")
axes.set_xlabel("Number of Bedrooms")

plt.show()
plt.close()

The distribution of the number of bedrooms appears to be right-skewed. This makes sense, as one or two bedrooms may be common for most homes, with 3 or more bedrooms being less commons in homes, especially in a densely-populated area.

### Accommodates
The `accommodates` variable is a numerical integer value. The `accommodates` variable represents the number of beds in the AirBnb. More beds generally corresponds to a higher capacity for overnight guests.

In [ ]:
b_df.accommodates.describe()

75% of AirBnB listings in the DC area are capable of accomodating 4 people. This unsurprisingly folows a similar pattern to both the `beds` and `bedroom` variables.

In [ ]:
h=1
mn = int(b_df["accommodates"].min())
mx = int(b_df["accommodates"].max())
bins = [i for i in range( mn, mx, h)]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.hist(b_df["accommodates"],bins=bins, histtype='bar', color="DimGray", ec='black')
axes.set_title("Accommodation distribution")
axes.set_xlabel("Number of Guests Accommodated")

plt.show()
plt.close()

The data are once again right-skewed. Even number `accommodates` values are more common, which makes sense given that one bed is often counted as accommodating 2 guests.

### Host Response Time

`host_response_time` is a categorical variable. More specifically, it is a variable that takes values of a few days or more, within a day, within a few hours, and within an hour. We can get a distribution of its values using `value_counts()`:

In [ ]:
eda_dkn["host_response_time"].value_counts().sort_index()

Interestingly, the dataset has the highest number of within an hour, the second highest number of within a few hours, the 3rd highest number of with a day and the lowest as a few days or more. This suggests that the host tends to respond quickly in order to rent out their houses.

### Host Response Rate

`host_response_rate` is a numerical variable that measures the percentages that the host will respond. We would expect that the price of the Airbnb will increase with the increase of the host_response_rate. Let's get a sense of the range of this variable by looking at the summary statistics:

In [ ]:
eda_dkn["host_response_rate"].describe()

The minimum `host_response_rate` is 0 which means that the host never responded and the maximum is 100%. This is a big range which tells me that the variable is definitely skewed. The mean is 96.9 and the median is 100 which suggests a left skewness. Recall that when the mean < median, this suggest a left- skewed distribution. Let's plot a histogram of the data using Freeman Diaconis bins. The use of Freeman Diaconis bins allows us to select the width of the bins to be used in a histogram. 

In [ ]:
# Ref: Fundamentals
mn = int(eda_dkn.host_response_rate.min())
mx = int(eda_dkn.host_response_rate.max())
bins = [i for i in range( mn, mx, 1)]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.hist(eda_dkn.host_response_rate,bins=bins, color="DimGray", edgecolor = "white")
axes.set_title("host_response_rate distribution")
axes.set_xlabel("host_response_rate")
axes.grid()

plt.show()
plt.close()

The distribution of the data looks very much skewed left which makes some sense given if the host responds more, the price will go up.

### Host Acceptance Rate

`host_acceptance_rate` is a numerical variable that measures percentages of host accepts the requests. Similarly, we should expect that the price would go up with the increase of the host_acceptance_rate. Let's get a sense of the range of this variable by looking at the summary statistics:

In [ ]:
eda_dkn["host_acceptance_rate"].describe()

The variable is definitely skewed left. The mean is 88.7% and the median is 97.0%. The 1st quartile is the same as the mean while the 3rd quartile is 100%. Let's plot a histogram of the data using Freeman Diaconis bins: 

In [ ]:
h = freeman_diaconis(eda_dkn["host_acceptance_rate"])
print("Freeman Diaconis: ", h)
mn = int(eda_dkn["host_acceptance_rate"].min())
mx = int(eda_dkn["host_acceptance_rate"].max())
bins = [i for i in range( mn, mx, h)]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.hist(eda_dkn["host_acceptance_rate"],bins=bins, histtype='bar', color="DimGray", ec='black')
axes.set_title("host_acceptance_rate distribution - Freeman Diaconis")
axes.set_xlabel("host_acceptance_rate")

plt.show()
plt.close()

The distribution of the data is skewed left. Similarly, if we assume that the price will depend on the increase of host_acceptance_rate, it created this skewness. 

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT id, AVG(number_of_reviews) as avg_rev
FROM listings
GROUP BY id;
'''

cur.execute(query)
con.commit()

number_of_reviews = cur.fetchall()

con.close()

In [ ]:
number_of_reviews = pd.DataFrame(number_of_reviews, columns=['ID', 'Average # of Reviews'])
number_of_reviews

In [ ]:
number_of_reviews['Average # of Reviews'].describe()

In [ ]:
h = freeman_diaconis(number_of_reviews ['Average # of Reviews'])
print('Freeman Diaconis: ', h)
mn = int(number_of_reviews['Average # of Reviews'].min())
mx = int(number_of_reviews['Average # of Reviews'].max())
bins = [i for i in range( mn, mx, h)]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.hist(number_of_reviews['Average # of Reviews'],bins=bins, color='gray')
axes.set_title('Average # of Reviews distribution - Freeman Diaconis')
axes.set_xlabel('Charge')

plt.show()
plt.close()

In [ ]:
figure = plt.figure(figsize=(10, 6)) # first element is width, second is height.

axes = figure.add_subplot(1, 1, 1)

axes.hist(number_of_reviews['Average # of Reviews'], color='gray', bins=20)
axes.set_ylabel('Count')
axes.set_xlabel('Reviews')
axes.set_title('Average Number of Reviews of an AirBnB Listing in Washington, DC')

plt.show()
plt.close()

### Review Scores Rating

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT id, AVG(review_scores_rating) AS FLOAT
FROM listings
GROUP BY id;
'''

cur.execute(query)
con.commit()

review_scores_rating = cur.fetchall()

con.close()

In [ ]:
review_scores_rating = pd.DataFrame(review_scores_rating, columns=['ID', 'Average Review Scores'])
review_scores_rating

In [ ]:
review_scores_rating['Average Review Scores'].describe()

In [ ]:
figure = plt.figure(figsize=(10, 6)) # first element is width, second is height.

axes = figure.add_subplot(1, 1, 1)

axes.hist(review_scores_rating['Average Review Scores'], color='dimgray', bins=20)
axes.set_ylabel('Count')
axes.set_xlabel('Review Scores')
axes.set_title('Average Review Scores of AirBnB Listings in Washington, DC')

plt.show()
plt.close()

### Reviews Per Month

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT id, AVG(reviews_per_month) AS FLOAT
FROM listings
GROUP BY id;
'''

cur.execute(query)
con.commit()

reviews_per_month = cur.fetchall()

con.close()

In [ ]:
reviews_per_month = pd.DataFrame(reviews_per_month, columns=['ID', 'Average Reviews Per Month'])
reviews_per_month

In [ ]:
reviews_per_month['Average Reviews Per Month'].describe()

### Room Type

room_type is orginally a text variable, but will be used as a categorical variable in our analysis.

All homes are grouped into the following three room types: Entire place, Private room, and Shared room.

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

sql_query = pd.read_sql_query('''SELECT room_type, property_type, instant_bookable, price
FROM listings''', con)

df = pd.DataFrame(sql_query, columns = ['room_type', 'price', 'instant_bookable', 'property_type'])
df

In [ ]:
# this changes price from currency to a float
df["price"] = df["price"].replace("[$,]", "", regex=True).astype(float)
df.loc[df["room_type"].str.contains('hotel'), "room_type"] = "Private room"

In [ ]:
df.room_type.value_counts()

Roughly 74.8% of the listings are categorized as entire home or apartment, 23.5% of listings are categorized as a private room, 1.4% of listings are categorized as a shared room, and less than 1% is categorized as a hotel room.

### Instant Bookable

instant_bookable is a binary categorical variable which tells us if a listing is available for instant booking (true) or not (false).

In [ ]:
df.instant_bookable.value_counts()

Approximately 69% of the observed listings are not available to instantly book, whereas 31% of listings are available to be instantly booked.

### Property Type

property_type is a unique text variable, but for this analysis will be used as a categorical variable.

In [ ]:
df.property_type.value_counts()

While this is a categorical variable, it seems some of the listing property types have unique names/titles that only relate to one or a few listings, causing a large number of buckets/options.

## Pairwise Variable EDA

### Price vs. Bathroom Type

Comparing the AirBnb listing price means that a numerical value (price) will be compared to a categorical variable (bathroom type). 

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT id, CAST(REPLACE(price, '$', '') AS FLOAT), bathrooms_text FROM listings'''

cur.execute(query)
con.commit()

price_bathroom = cur.fetchall()
con.close()

In [ ]:
listing_id = []
price = []
bath = []

for pb in price_bathroom:
    listing_id.append(pb[0])
    price.append(pb[1])
    bath.append(bathroom_mapping[pb[2]])
price_bath_df = pd.DataFrame({'listing_id': listing_id,
                             'price': price,
                             'bath': bath})

grouped_price_bath = price_bath_df.groupby('bath')
grouped_price_bath = grouped_price_bath['price'].describe()

t = tabulate(grouped_price_bath, 
             headers=('Bathroom Type', 'Count', 'Mean', 'Stdev', 'Min', '25%', '50%', '75%', 'Max'),
            tablefmt='fancy_grid')
print(t)
print('\n')

The IQR differs between the bathroom types available in the listings. AirBnB listings with shared bathrooms have similar nightly prices, with median values ranging from 52-58 USD per night. For listings that have private bathrooms, there is a significant difference in the meidan values. The median nightly cost of an AirBnB listing with 1 private bathroom is 120 USD per night. For listings with two private bathrooms, the nightly cost jumps to 225 USD. The median for listings with three or more private bathrooms is 300 USD per night. 

In [ ]:
figure = plt.figure(figsize=(10, 6))
axes = figure.add_subplot(1, 1, 1)
grouped_price_bath = price_bath_df.groupby('bath')
price_data = [grouped_price_bath.get_group(bath)['price'] for bath in grouped_price_bath.groups]

patch = axes.boxplot(price_data, labels=grouped_price_bath.groups.keys(), patch_artist=True, zorder=1)

for i, bath in enumerate(grouped_price_bath.groups.keys()):
    subdata = grouped_price_bath.get_group(bath)['price']
    x = np.random.normal(i + 1, 0.01, size=len(subdata))
    axes.plot(x, subdata, 'o', alpha=0.4, color="DimGray", zorder=2)

axes.set_xlabel('Bath')
axes.set_ylabel('Price')
axes.set_title('Distribution of Price, Grouped by Bath')
plt.show()
plt.close()

For listings with private bathrooms, the prices tend to increase with more bathrooms. Listings that have shared bathrooms appear to have the same values, regardless of how many shared baths are available.

### Price vs. Neighborhood Group

To compare the listing price to the neighborhood group, the IQR needs to be calculated for each of the 39 neighborhood groupings.

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT id, CAST(REPLACE(price, '$', '') AS FLOAT), neighbourhood_cleansed FROM listings'''

cur.execute(query)
con.commit()

nbh_price = cur.fetchall()
con.close()

In [ ]:
listing_id = []
price = []
nbhd = []

for pb in nbh_price:
    listing_id.append(pb[0])
    price.append(pb[1])
    nbhd.append(pb[2])
    
nbhd_df = pd.DataFrame({'listing_id': listing_id,
                             'price': price,
                             'nbhd': nbhd})

nbhd_grouped = nbhd_df.groupby('nbhd')
nbhd_grouped = nbhd_grouped['price'].describe()
nbhd_grouped= nbhd_grouped.sort_values('count', ascending=False)

print(nbhd_grouped)

The IQR differs between the bathroom types available in the listings. AirBnB listings with shared bathrooms have similar nightly prices, with median values ranging from 52-58 USD per night. For listings that have private bathrooms, there is a significant difference in the meidan values. The median nightly cost of an AirBnB listing with 1 private bathroom is 120 USD per night. For listings with two private bathrooms, the nightly cost jumps to 225 USD. The median for listings with three or more private bathrooms is 300 USD per night. 

Some neighborhoods tend to have higher median AirBnb Listing prices. Limiting the comparison to the neighborhoods that have at least 300 AirBnB listings:

In [ ]:
nbhd_grouped = nbhd_grouped[nbhd_grouped['count']>300]
print(nbhd_grouped)

The "Brightwood Park, Crestwood, Petworth" neighborhoods appear to have lower nightly costs of AirbnB listings. In contrast, the "Georgetown, Burleith/Hillandale" area appears to have the highest nightly costs for AirBnbs.

In [ ]:
nbhd_groups = nbhd_grouped.index

In [ ]:
figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)

n_list = nbhd_grouped.index.unique()
nbhd_df.loc[~nbhd_df['nbhd'].isin(nbhd_groups), 'nbhd'] = 'Other'

labels = pd.unique(nbhd_df['nbhd'].values)
grouped_data = [nbhd_df["price"].loc[nbhd_df['nbhd'] == k] for k in labels]
patch = axes.boxplot(grouped_data, labels=labels, patch_artist=True, zorder=1)

for i, k in enumerate(labels):
    subdata = nbhd_df["price"].loc[nbhd_df['nbhd'] == k]
    x = np.random.normal(i + 1, 0.01, size=len(subdata))
    axes.plot(x, subdata, 'o', alpha=0.4, color="DimGray", zorder=2)

axes.set_xlabel("Neighborhood")
axes.set_ylabel("Price")
axes.set_title("Distribution of Price, Grouped By Neighborhood")
axes.set_xticklabels(labels, rotation=90)
plt.show()
plt.close()

The IQR of the pricing by neighborhood suggests that there is not a massive difference in prices of AirBnb listings depending on neighborhood. It does appear that the "Brightwood Park, Crestwood, Petworth" neighborhoods have an overall lower nightly cost.

### Price vs. Superhost Status

Around 30% of the AirBnB listings have hosts who also possess the superhost status. This status is advertised as AirBnb as being a factor in providing hosts with more success in bookings. This could mean that the hosts are able to list properties at a higher price.

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

query = '''SELECT id, CAST(REPLACE(price, '$', '') AS FLOAT), host_is_superhost
FROM listings;
'''

cur.execute(query)
con.commit()

super_price = cur.fetchall()
con.close()

In [ ]:
listing_id = []
superhost = []
price = []

for pb in super_price:
    listing_id.append(pb[0])
    price.append(pb[1])
    if pb[2] == '':
        superhost.append('f')
    else:
        superhost.append(pb[2])
super_df = pd.DataFrame({'listing_id': listing_id,
                             'price': price,
                             'superhost': superhost})

super_df['superhost'] = super_df['superhost'].replace({'t': 'Superhost',
                                                    'f': 'Not Superhost'})

grouped_super = super_df.groupby('superhost')
grouped_super = grouped_super['price'].describe()

t = tabulate(grouped_super, 
             headers=('Is Superhost?', 'Count', 'Mean', 'Stdev', 'Min', '25%', '50%', '75%', 'Max'),
            tablefmt='fancy_grid')
print(t)
print('\n')

The IQR values for superhosts and regular hosts are similar, with some minor variations. There does not to appear a significant difference between the price of a listing and the host's "superhost" status.

### Number of Bedrooms vs. Price

In [ ]:
lowess = sm.nonparametric.lowess(b_df["price"], b_df["bedrooms"], frac=.3)
lowess_x = list(zip(*lowess))[0]
lowess_y = list(zip(*lowess))[1]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.scatter( b_df[ "bedrooms"], b_df["price"], marker="o", color="DimGray")
axes.plot(lowess_x, lowess_y, color="DarkRed")

axes.set_title("Plot of Bedrooms v. Price with LOWESS")
axes.set_xlabel( "Bedrooms")
axes.set_ylabel( "Price")

plt.show()
plt.close()

In [ ]:
correlation(b_df, 'bedrooms', 'price')

### Beds v. Price with LOWESS

In [ ]:
lowess = sm.nonparametric.lowess(b_df["price"], b_df["beds"], frac=.3)
lowess_x = list(zip(*lowess))[0]
lowess_y = list(zip(*lowess))[1]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.scatter( b_df[ "beds"], b_df["price"], marker="o", color="DimGray")
axes.plot(lowess_x, lowess_y, color="DarkRed")

axes.set_title("Plot of Beds v. Price with LOWESS")
axes.set_xlabel( "Beds")
axes.set_ylabel( "Price")

plt.show()
plt.close()

In [ ]:
correlation(b_df, 'beds', 'price')

### Accommodates v. Price with LOWESS

In [ ]:
lowess = sm.nonparametric.lowess(b_df["price"], b_df["accommodates"], frac=.3)
lowess_x = list(zip(*lowess))[0]
lowess_y = list(zip(*lowess))[1]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.scatter( b_df[ "accommodates"], b_df["price"], marker="o", color="DimGray")
axes.plot(lowess_x, lowess_y, color="DarkRed")

axes.set_title("Plot of Accommodates v. Price with LOWESS")
axes.set_xlabel( "Accommodates")
axes.set_ylabel( "Price")

plt.show()
plt.close()

In [ ]:
correlation(b_df, 'accommodates', 'price')

### Price vs Host Response Rate

We can definitely expect to see the higher the host_response_rate, the higher the price is. Since this pairwise comparison is between two numerical variables, we can start by seeing the correlation of the two variables. 

In [ ]:
correlation(eda_dkn, "host_response_rate", "price")

Pearson's correlation coefficient, $r$, is only 0.003 which indicates a very weak positive relationship. Spearman's is at 0.032 which might indicate a very weak monotonic relationship. Let's plot the data using Lowess. That is by using locally weighted smoothing (LOWESS), this method of regression analysis creates a smooth line through a scatterplot in order to see the trend between both variables. 

In [ ]:
lowess = sm.nonparametric.lowess(eda_dkn["price"], eda_dkn["host_response_rate"], frac=.3)
lowess_x = list(zip(*lowess))[0]
lowess_y = list(zip(*lowess))[1]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.scatter( eda_dkn["host_response_rate"], eda_dkn["price"], marker="o", color="DimGray")
axes.plot(lowess_x, lowess_y, color="DarkRed")

axes.set_title("Plot of host_response_rate v. price with LOWESS")
axes.set_xlabel( "host_response_rate")
axes.set_ylabel( "price")

plt.show()
plt.close()

Through this scatterplot and the lowess line, we can see that there is a very weak postive linear relationship between host_response_rate and price. 

### Price vs Host Acceptance Rate

We can expect to see the higher the host_acceptance_rate, the higher the price is. Since this pairwise comparison is between two numerical variables, we can start by seeing the correlation of the two variables. 

In [ ]:
correlation(eda_dkn, "host_acceptance_rate", "price")

Pearson's correlation coefficient, $r$, is only 0.088 which indicates a very weak positive relationship. Spearman's is at 0.144 which might indicate a very weak monotonic relationship. Let's plot the data using Lowess. That is by using locally weighted smoothing (LOWESS), this method of regression analysis creates a smooth line through a scatterplot in order to see the trend between both variables. 

In [ ]:
import statsmodels.api as sm

lowess = sm.nonparametric.lowess(eda_dkn["price"], eda_dkn["host_acceptance_rate"], frac=.3)
lowess_x = list(zip(*lowess))[0]
lowess_y = list(zip(*lowess))[1]

figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.scatter( eda_dkn["host_acceptance_rate"], eda_dkn["price"], marker="o", color="DimGray")
axes.plot(lowess_x, lowess_y, color="DarkRed")

axes.set_title("Plot of host_acceptance_rate v. price with LOWESS")
axes.set_xlabel( "host_acceptance_rate")
axes.set_ylabel( "price")

plt.show()
plt.close()

Through this scatterplot and the lowess line, we can see that there is a very weak postive linear relationship between host_response_rate and price. 

### Price vs Host Response Time

It is possible that the host_response_time has a negative correlation with the price since when the not hearing from the host, most people just start to look for a new ad. Because this pairwise correlation is marked by a categorical variable and a numerical variable, using summary statistics in relation to each other gives us the perspective we are looking for:

In [ ]:
# describe_by_category(eda_dkn, "price", "host_response_time", transpose=True)

grouped_hot_response_rate = eda_dkn[['host_response_time', 'price']].groupby('host_response_time')
grouped_hot_response_rate_stats = grouped_hot_response_rate['price'].describe()
grouped_hot_response_rate_stats

### Number of Reviews vs. Review Scores Rating

In [ ]:
print('r   = ', stats.pearsonr(number_of_reviews['Average # of Reviews'], review_scores_rating['Average Review Scores'])[0])
print('rho = ', stats.spearmanr(number_of_reviews['Average # of Reviews'], review_scores_rating['Average Review Scores'])[0])

In [ ]:
figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.scatter(review_scores_rating['Average Review Scores'], number_of_reviews['Average # of Reviews'], marker='o', color='gray')

axes.set_ylabel('# of Reviews')
axes.set_xlabel('Review Scores')
axes.set_title('Scatter Plot of # of Reviews vs. Review Scores')

plt.show()
plt.close()

In [ ]:
figure = plt.figure(figsize=(10, 10))

axes = figure.add_subplot(2, 1, 1)
axes.hist(number_of_reviews['Average # of Reviews'],bins=bins,color="gray")
axes.set_title("Average # of Reviews distribution - Freeman Diaconis")
axes.set_xlabel("Average # of Reviews")

axes = figure.add_subplot(2, 1, 2)
axes.scatter(number_of_reviews['Average # of Reviews'], review_scores_rating['Average Review Scores'], marker="o", color="gray")

axes.set_ylabel("Average Review Scores")
axes.set_xlabel("Average # of Reviews")
axes.set_title("Scatter Plot of Average # of Reviews vs. Average Review Scores")

plt.show()
plt.close()

### Number of Reviews vs. Reviews Per Month

In [ ]:
print('r   = ', stats.pearsonr(number_of_reviews['Average # of Reviews'], reviews_per_month['Average Reviews Per Month'])[0])
print('rho = ', stats.spearmanr(number_of_reviews['Average # of Reviews'], reviews_per_month['Average Reviews Per Month'])[0])

In [ ]:
figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.scatter(reviews_per_month['Average Reviews Per Month'], number_of_reviews['Average # of Reviews'], marker='o', color='gray')

axes.set_ylabel('# of Reviews')
axes.set_xlabel('Reviews Per Month')
axes.set_title('Scatter Plot of # of Reviews vs. Reviews Per Month')

plt.show()
plt.close()

In [ ]:
figure = plt.figure(figsize=(10, 10))

axes = figure.add_subplot(2, 1, 1)
axes.hist(number_of_reviews['Average # of Reviews'],bins=bins,color="gray")
axes.set_title("Average # of Reviews distribution - Freeman Diaconis")
axes.set_xlabel("Average # of Reviews")

axes = figure.add_subplot(2, 1, 2)
axes.scatter(number_of_reviews['Average # of Reviews'], reviews_per_month['Average Reviews Per Month'], marker="o", color="gray")

axes.set_ylabel("Average Reviews Per Month")
axes.set_xlabel("Average # of Reviews")
axes.set_title("Scatter Plot of Average # of Reviews vs. Average Reviews Per Month")

plt.show()
plt.close()

### Review Scores Rating vs. Reviews Per Month

In [ ]:
print('r   = ', stats.pearsonr(review_scores_rating['Average Review Scores'], reviews_per_month['Average Reviews Per Month'])[0])
print('rho = ', stats.spearmanr(review_scores_rating['Average Review Scores'], reviews_per_month['Average Reviews Per Month'])[0])

In [ ]:
figure = plt.figure(figsize=(10, 6))

axes = figure.add_subplot(1, 1, 1)
axes.scatter(reviews_per_month['Average Reviews Per Month'], review_scores_rating['Average Review Scores'], marker='o', color='gray')

axes.set_ylabel('Average Review Scores')
axes.set_xlabel('Reviews Per Month')
axes.set_title('Scatter Plot of Average Review Scores vs. Reviews Per Month')

plt.show()
plt.close()

In [ ]:
figure = plt.figure(figsize=(10, 10))

axes = figure.add_subplot(2, 1, 1)
axes.hist(review_scores_rating['Average Review Scores'], color='gray', bins=20)
axes.set_ylabel('Count')
axes.set_xlabel('Average Review Scores')
axes.set_title('Average Review Scores distribution')

axes = figure.add_subplot(2, 1, 2)
axes.scatter(review_scores_rating['Average Review Scores'], reviews_per_month['Average Reviews Per Month'], marker="o", color="gray")

axes.set_ylabel("Average Reviews Per Month")
axes.set_xlabel("Average Review Scores")
axes.set_title("Scatter Plot of Average Review Scores vs. Average Reviews Per Month")

plt.show()
plt.close()

While this is a categorical variable, it seems some of the listing property types have unique names/titles that only relate to one or a few listings, causing a large number of buckets/options.

### Price vs Room Type

In [ ]:
con=sqlite3.connect(db_path)
cur = con.cursor()

sql_query = pd.read_sql_query('''SELECT room_type, property_type, instant_bookable, price
FROM listings''', con)

df = pd.DataFrame(sql_query, columns = ['room_type', 'price', 'instant_bookable', 'property_type'])
df

df["price"] = df["price"].replace("[$,]", "", regex=True).astype(float)

In [ ]:
grouped_price_room_type = df[['room_type', 'price']].groupby('room_type')
grouped_price_room_type_stats = grouped_price_room_type['price'].describe()
grouped_price_room_type_stats


table = tabulate(grouped_price_room_type_stats, 
             headers=('Room Type', 'Count', 'Mean', 'Stdev', 'Min', '25%', '50%', '75%', 'Max'),
            tablefmt='fancy_grid')
print(table)
print('\n')

Entire homes/apartments are on average the most expensive (202.20), followed by a private room (123.78). The least expensive stay is a shared room (54.02). The range for entire home/apartments is significantly larger/wider than the other options, ranging from 10 to 7500. This difference tells us that there are surely some extreme values in our dataset.  


### Price vs Instant Bookable

In [ ]:
grouped_price_instant_bookable = df[['instant_bookable', 'price']].groupby('instant_bookable')
grouped_price_instant_bookable_stats = grouped_price_instant_bookable['price'].describe()
grouped_price_instant_bookable_stats


table2 = tabulate(grouped_price_instant_bookable_stats, 
            headers=('Instant Bookable', 'Count', 'Mean', 'Stdev', 'Min', '25%', '50%', '75%', 'Max'),
            tablefmt='fancy_grid')
print(table2)
print('\n')

The average price of an Air BnB listing is higher for those that are instantly bookable (191.53) compared to listings that are not instantly bookable (176.21). Interestingly, the minimum price for a instantly bookable stay is 16, whereas the non-instantly bookable stays do not have a minimum.  


### Price vs Property Type

In [ ]:
property_type_df = df[['property_type','price']]
#property_type_df

In [ ]:
df_2 = pd.DataFrame(property_type_df)
df_2['property_type_2'] = '0'

df_2.loc[df_2["property_type"].str.contains('condo|loft'), "property_type_2"] = "Condo"
df_2.loc[df_2["property_type"].str.contains('apartment'), "property_type_2"] = "Apartment"
df_2.loc[df_2["property_type"].str.contains('townhouse'), "property_type_2"] = "Townhouse"
df_2.loc[df_2["property_type"].str.contains('home|place'), "property_type_2"] = "Home"
df_2.loc[df_2["property_type"].str.contains('guest suite|guesthouse'), "property_type_2"] = "Guest Suite/Guesthouse"
df_2.loc[df_2["property_type"].str.contains('rental'), "property_type_2"] = "Rental Unit"
df_2.loc[df_2["property_type"].str.contains('hotel|resort'), "property_type_2"] = "Hotel Room"
df_2.loc[df_2["property_type"].str.contains('hostel'), "property_type_2"] = "Hostel"
df_2.loc[df_2["property_type"].str.contains('villa|bungalow'), "property_type_2"] = "Villa/Bungalow"
df_2.loc[df_2["property_type"].str.contains('Cottage|cottage|Boat|Camper/RV|Campsite|Castle|Floor|Houseboat|Tent|Tower'), "property_type_2"] = "Unique Stay"
df_2.loc[df_2["property_type"].str.contains('Tiny|tiny'), "property_type_2"] = "Tiny Home"
df_2.loc[df_2["property_type"].str.contains('Breakfast|breakfast'), "property_type_2"] = "Bed & Breakfast"
df_2.loc[df_2["property_type"].str.contains('particular|aparthotel'), "property_type_2"] = "Other"
df_2.loc[df_2["property_type_2"].str.contains('0'), "property_type_2"] = "Other"

df_2.loc[df_2["property_type"].str.contains('Entire'), "property_type"] = "Entire Space"
df_2.loc[df_2["property_type"].str.contains('Private|private|apartment|breakfast'), "property_type"] = "Private Room"
df_2.loc[df_2["property_type"].str.contains('Shared|shared'), "property_type"] = "Shared Room"
df_2.loc[df_2["property_type"].str.contains('Hotel|hotel'), "property_type"] = "Hotel"
df_2.loc[df_2["property_type"].str.contains('Hostel|hostel'), "property_type"] = "Hostel"
df_2.loc[df_2["property_type"].str.contains('Boat|Camper/RV|Campsite|Casa particular|Castle|Floor|Houseboat|Tent|Tiny home|Tower'), "property_type"] = "Unique Stay"
#df_2

In [ ]:
grouped_price_property_type = df_2[['property_type', 'price', 'property_type_2']].groupby('property_type')
grouped_price_property_type_stats = grouped_price_property_type['price'].describe().reset_index()
#grouped_price_property_type_stats

In [ ]:
grouped_price_property_type_2 = df_2[['property_type', 'price', 'property_type_2']].groupby('property_type_2')
grouped_price_property_type_stats_2 = grouped_price_property_type_2['price'].describe().reset_index()
grouped_price_property_type_stats_2

We grouped the different property types into overall categories (for example: "Apartment" includes rentals that are the entire apartment, private room of an apartment, and shared room in an apartment). There were originally 53 different categories, and we were able to pair that down to about 12 groups. Unique stays include boats, boathouse, campsite, castle, tent, camper/rv, cottage, tower, and a few other listing types. Based on the analysis results, the highest average cost is actually for hotel rooms (382.43) and the least expensive stay is a hostel (84.76). Interestingly, the price range for a home stay (10, 7500) and townhouse stay (20, 4357) seem fairly extreme, leading us to believe that there are some extreme values in our dataset. It is worth looking further into this variable and how it interacts with other variables such as number of rooms and location to see how those variables may influence the overall price of a listing. 